In [ ]:
import pandas as pd
import sqlite3 as sq
import numpy as np

In [ ]:
#2 подключаюсь к бд
con=sq.connect("../data/checking-logs.sqlite")

In [ ]:
#3 получаю схему таблицы pageviews с помощью pd.io.sql.read_sql и запроса PRAGMA table_info(pageviews)
pd.io.sql.read_sql("PRAGMA table_info(pageviews)", con)

In [ ]:
#4 Получаю только первые 10 строк таблицы pageviews, чтобы проверить, как она выглядит
pd.io.sql.read_sql("SELECT * FROM (pageviews) LIMIT 10", con)

In [ ]:
#5 Получаю подтаблицу 
pageviews = pd.io.sql.read_sql("SELECT datetime, uid FROM pageviews WHERE uid LIKE 'user_%' ORDER by uid", con, index_col='datetime', parse_dates=['datetime'])
pageviews.head(5)

In [ ]:
#2 получаю схему таблицы checker
pd.io.sql.read_sql("PRAGMA table_info(checker)", con)

In [ ]:
#3 первые 10 строк checker
pd.io.sql.read_sql("SELECT * FROM (checker) LIMIT 10", con)

In [ ]:
#4 Подсчитываю, сколько строк удовлетворяют следующим условиям
pd.io.sql.read_sql("""SELECT COUNT (uid) FROM pageviews WHERE uid IN (SELECT uid FROM checker WHERE status='ready' AND numTrials=1 AND 
                               labname IN ('project1', 'laba04', 'laba04s', 'laba05', 'laba06', 'laba06s'))""", con)

In [ ]:
from pandas import DataFrame

In [ ]:
#2 Создаю новую таблицу datamart в базе данных, объединив таблицы pageviews и checker
datamart = pd.io.sql.read_sql("""
SELECT ch.uid, ch.labname, ch.timestamp AS first_commit_ts,
MIN(page.datetime) AS first_view_ts
FROM checker ch
LEFT JOIN pageviews page
ON ch.uid=page.uid
WHERE ch.status='ready' AND ch.numTrials=1 AND ch.labname IN ('laba04','laba04s','laba05','laba06','laba06s','project1')AND ch.uid LIKE 'user_%'
GROUP BY ch.timestamp, ch.uid;""", 
con, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart.to_sql('datamart', con, if_exists='replace')  

In [ ]:
#3 Создаю новую таблицу datamart в базе данных, объединив таблицы pageviews и checker
#убираю все пропущенные в first_view_ts
test = datamart.dropna().copy()
#использую пропущенные в first_view_ts
control = datamart[datamart.first_view_ts.isna()].copy()
#заменяю пропущенные значения в control средним значением first_view_ts пользователей из test
control.first_view_ts = test.first_view_ts.mean()
test.to_sql('test', con, if_exists='replace')
control.to_sql('control', con, if_exists='replace')

In [ ]:
#2 схема таблицы test
pd.io.sql.read_sql("PRAGMA table_info(test)", con)

In [ ]:
#3 первые 10 строк таблицы test
pd.io.sql.read_sql("SELECT * FROM (test) LIMIT 10", con)

In [ ]:
import datetime
from datetime import date, timedelta

In [ ]:
#4 стоит рассматривать значение по модулю
#Найди среди всех пользователей минимальное значение этой самой дельты (периода времени от даты первого коммита пользователя до срока сдачи соответствующего лабораторного задания)
df_min = pd.io.sql.read_sql("""
SELECT tx.uid, CAST(MIN(JULIANDAY(tx.first_commit_ts) - JULIANDAY(dl.deadlines, 'unixepoch'))*24 AS INTEGER) AS 'MIN(diff)'
FROM test  tx
LEFT JOIN deadlines dl
ON tx.labname = dl.labs
WHERE tx.labname IN ('laba04','laba04s','laba05','laba06','laba06s');""", 
                            con)
df_min.to_sql('df_min', con, if_exists='replace')
df_min

In [ ]:
#5 Выполни те же самые операции для максимального значения дельты, используя только один запрос. Название итогового датафрейма — df_max
df_max = pd.io.sql.read_sql("""
SELECT tx.uid, CAST(MAX(JULIANDAY(tx.first_commit_ts) - JULIANDAY(dl.deadlines, 'unixepoch'))*24 AS INTEGER) AS 'MAX(diff)'
FROM test  tx
LEFT JOIN deadlines dl
ON tx.labname = dl.labs
WHERE tx.labname IN ('laba04','laba04s','laba05','laba06','laba06s');""", 
                            con)
df_max.to_sql('df_max', con, if_exists='replace')
df_max

In [ ]:
#6 Выполни те же самые операции для среднего значения дельты, используя только один запрос. На этот раз ваш итоговый датафрейм не должен включать столбец uid; имя датафрейма — df_avg.
df_avg  = pd.io.sql.read_sql("""
SELECT CAST(AVG(JULIANDAY(tx.first_commit_ts) - JULIANDAY(dl.deadlines, 'unixepoch'))*24 AS INTEGER) AS 'avg(diff)'
FROM test  tx
LEFT JOIN deadlines dl
ON tx.labname = dl.labs
WHERE tx.labname IN ('laba04','laba04s','laba05','laba06','laba06s');""", 
                            con)
df_avg.to_sql('df_avg', con, if_exists='replace')
df_avg

In [ ]:
#7
#SQL оператор DISTINCT используется для удаления дубликатов из результирующего набора оператора SELECT.
views_diff = pd.io.sql.read_sql("""
SELECT DISTINCT t.uid, t.avg_diff, p.pageviews 
FROM (SELECT uid, AVG(cast((JULIANDAY(test.first_commit_ts) - 
            JULIANDAY(dl.deadlines, 'unixepoch'))*24 AS integer)) AS avg_diff
            FROM test
            LEFT JOIN deadlines dl
            ON test.labname = dl.labs
            WHERE test.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s')
            GROUP BY uid) 
AS t
LEFT JOIN (SELECT uid, COUNT(datetime) AS pageviews
FROM pageviews
GROUP BY uid) AS p
ON t.uid = p.uid""", con, index_col='uid')
views_diff.to_sql('df_avg', con, if_exists='replace')
views_diff  

In [ ]:
# Предназначение корреляционного анализа сводится к выявлению наличия зависимости между различными факторами. 
# Коэффициент корреляции принимает значения от -1 до+ 1. 
# Положительное значение коэффициента корреляции свидетельствует о наличии прямой связи, отрицательное – обратной.
views_diff.corr()

In [ ]:
# первый просмотр ленты наименьшее значение min_view
# сначала проверяем пользователей которые сдали задание до первого просмотра test.first_commit_ts < min_view.min_view
# потом проверяем пользователей которые сдали после первого просмотра test.first_commit_ts > min_view.min_view
# запрос из двух частей, после они соединяются с помощью команды union, она дополняет таблицу после первого запроса

test_results = pd.io.sql.read_sql("""
SELECT  times, AVG(diff)
FROM
     (SELECT uid, 'before' AS 'times', (diff)
      FROM 
          (SELECT uid, AVG(cast((JULIANDAY(test.first_commit_ts) - JULIANDAY(dl.deadlines, 'unixepoch'))*24 AS integer)) AS diff 
           FROM test 
           LEFT JOIN deadlines AS dl, 
                (SELECT uid AS id, min(first_view_ts) AS min_view
                FROM test
                GROUP BY uid) AS min_view
           ON test.labname = dl.labs AND test.uid = min_view.id 
           WHERE test.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s') 
                AND test.first_commit_ts < min_view.min_view
                GROUP BY uid)
UNION
SELECT uid, 'after' AS 'times', (diff)
FROM 
    (SELECT uid, AVG(cast((JULIANDAY(test.first_commit_ts) - JULIANDAY(dl.deadlines, 'unixepoch'))*24 AS integer)) AS diff 
     FROM test 
     LEFT JOIN deadlines AS dl, 
                (SELECT uid AS id, min(first_view_ts) AS min_view
                FROM test
                GROUP BY uid) AS min_view
     ON test.labname = dl.labs AND test.uid = min_view.id 
     WHERE test.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s') 
                AND test.first_commit_ts > min_view.min_view AND test.uid IN 
                    (SELECT uid 
                    FROM test 
                    LEFT JOIN deadlines AS dl, 
                        (select uid AS id, min(first_view_ts) AS min_view
                        FROM test
                        GROUP BY uid) AS min_view
                    ON test.labname = dl.labs AND test.uid = min_view.id 
                    WHERE test.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s')
                        AND test.first_commit_ts < min_view.min_view
                    GROUP BY uid)
                GROUP BY uid))
        group by times""",
            con)
test_results

In [ ]:
control_results = pd.io.sql.read_sql("""
SELECT  times, AVG(diff)
FROM
     (SELECT uid, 'before' AS 'times', (diff)
      FROM 
          (SELECT uid, AVG(cast((JULIANDAY(control.first_commit_ts) - JULIANDAY(dl.deadlines, 'unixepoch'))*24 AS integer)) AS diff 
           FROM control 
           LEFT JOIN deadlines AS dl, 
                (SELECT uid AS id, min(first_view_ts) AS min_view
                FROM control
                GROUP BY uid) AS min_view
           ON control.labname = dl.labs AND control.uid = min_view.id 
           WHERE control.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s') 
                AND control.first_commit_ts < min_view.min_view
                GROUP BY uid)
UNION
SELECT uid, 'after' AS 'times', (diff)
FROM 
    (SELECT uid, AVG(cast((JULIANDAY(control.first_commit_ts) - JULIANDAY(dl.deadlines, 'unixepoch'))*24 AS integer)) AS diff 
     FROM control 
     LEFT JOIN deadlines AS dl, 
                (SELECT uid AS id, min(first_view_ts) AS min_view
                FROM control
                GROUP BY uid) AS min_view
     ON control.labname = dl.labs AND control.uid = min_view.id 
     WHERE control.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s') 
                AND control.first_commit_ts > min_view.min_view AND control.uid IN 
                    (SELECT uid 
                    FROM control 
                    LEFT JOIN deadlines AS dl, 
                        (select uid AS id, min(first_view_ts) AS min_view
                        FROM control
                        GROUP BY uid) AS min_view
                    ON control.labname = dl.labs AND control.uid = min_view.id 
                    WHERE control.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s')
                        AND control.first_commit_ts < min_view.min_view
                    GROUP BY uid)
                GROUP BY uid))
        group by times""",
            con)
control_results

In [ ]:
con.close()

##  Дайте ответ: оказалось ли предположение верным и влияет ли наличие страницы с новостной лентой на поведение учащихся?

Да, влияет: модуль разности между временем от даты первого коммита пользователя до срока сдачи соответствующего лабораторного задания отличается у тестовой группы. 